In [8]:
import numpy as np
import sqlite3
import gps_geometry as gps

# Use only the first time
conn = sqlite3.connect("mydatabase.db") # or use :memory: to put it in RAM
cursor = conn.cursor()


In [9]:
# Read keywords to track from track.csv into variable trackstring

import csv
with open('track.csv', newline='') as csvfile:
    trackreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in trackreader:
        trackstring = ', '.join(row)
        print(','.join(row))

BMW, daimler, mercedes benz, AMG, audi


In [16]:
# The main body of the streamer class, writes to SQL

from twython import TwythonStreamer  
import json

# Filter out unwanted data
def process_tweet(tweet):  
    
      
    tweet_id_str = tweet['id_str']
    tweet_text = tweet['text']
    tweet_created_at = tweet['created_at']
    tweet_user_idstr= tweet['user']['id_str']
    tweet_hash_tags = str(tweet['entities']['hashtags'])
    
     
    if(tweet["geo"]!= None):
        tweet_geo_coords = tweet["geo"]["coordinates"]
        tweet_geo_lat =  tweet_geo_coords[0]
        tweet_geo_lon =  tweet_geo_coords[1]
    else:
        tweet_geo_coords = None
        tweet_geo_lon = None
        tweet_geo_lat = None
        
    #print("geo",tweet_geo_coords)
    #print("coordinates",tweet["coordinates"]) 
    #print(tweet_hash_tags)
    
    # User information
    tweet_user_id_str = tweet['user']['id_str']
    tweet_user_screen_name =  tweet['user']['screen_name']
    tweet_user_location = tweet['user']['location']
    tweet_user_url =  tweet['user']['url']
    tweet_user_description = tweet['user']['description']
    tweet_user_created_at = tweet['user']['created_at']
    tweet_user_followers_count =tweet['user']['followers_count']
    tweet_user_friends_count =tweet['user']['friends_count']
    tweet_user_statuses_count =tweet['user']['statuses_count']
    tweet_user_time_zone = tweet['user']['time_zone']
    
    print("Time Zone: ", tweet_user_time_zone)
    
    
     # Tweet Data: (8 items)
    # tweet_log_id, tweet_id_text, tweet_hashtag , tweet_text, created_at,  geo_lat,  geo_long , user_id_text
    d = [tweet_id_str , tweet_hash_tags,tweet_text,tweet_created_at,tweet_geo_lat,tweet_geo_lon, tweet_user_idstr]
    

    # User Data: (10 items) 
    # user_id_text, screen_name, location, url,  description text,
    # created_at, followers_count ,  friends_count ,statuses_count , time_zone 
    d2 = [tweet_user_id_str, tweet_user_screen_name,tweet_user_location,tweet_user_url,
          tweet_user_description, tweet_user_created_at,tweet_user_followers_count ,tweet_user_friends_count,
         tweet_user_statuses_count,tweet_user_time_zone]
    
    # Tweet log data: (6 items)
    # tweet_log_id, query text, geo_lat, geo_long, radius, timestamp
    
    return [d, d2]


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer): 
    
    # The class constructor
    def __init__(self,app_key, app_secret, oauth_token, oauth_token_secret,queryText='',long='',lat='',Radius=''):
        TwythonStreamer.__init__(self,app_key, app_secret, oauth_token, oauth_token_secret)
        self.queryText = queryText
        self.long = long
        self.lat = lat
        self.Radius = Radius

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            print('query info:',self.queryText,self.lat,self.long)
            tweet_data = process_tweet(data)
            tweet_id_text =  data['id_str']
            user_id = data['user']['id_str']
            time_stamp = data['created_at']
            self.save_to_sql(tweet_data, tweet_id_text, user_id, time_stamp)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_sql(self, tweet, tweet_id_text, user_id, time_stamp):
        
        #tweet_log_id = tweet_id_text
        query_text = self.queryText
        query_long = self.long
        query_lat = self.lat
        query_radius = self.Radius
        
        # tweet_log_id, query text, geo_lat, geo_long, radius, timestamp
        query_data = [tweet_id_text,query_text,query_lat,query_long,query_radius, time_stamp]
        
        print(query_data)
        
        
        query ='''
        SELECT count(*) FROM user WHERE user_id_text=?
        '''
        #cursor.execute("SELECT count(*) FROM user WHERE user_id_text=?", (user_id,))
        cursor.execute(query, (user_id,))
        
        
        # Save the User Data, if the user is Unique
        if cursor.fetchone()[0] == 0:
            
            usr_query ='''
            INSERT INTO user(user_id_text, screen_name, location, url, description, created_at,
            followers_count, friends_count, statuses_count, time_zone) VALUES (?,?,?,?,?,?,?,?,?,?)
            '''
            cursor.execute(usr_query,tweet[1])
            
        tweet_query='''
        INSERT INTO tweet(tweet_id_text,tweet_hashtag,tweet_text,created_at,
        geo_lat, geo_long, user_id_text) VALUES (?,?,?,?,?,?,?)
        '''
            
        log_query='''
        INSERT INTO tweet_log(tweet_id_text,query,geo_lat,geo_long,radius,timestamp_at) VALUES (?,?,?,?,?,?)
        '''
            
        # Insert the items into the appropriate table
        cursor.execute(tweet_query,tweet[0])
        cursor.execute(log_query, query_data)
        conn.commit()

In [17]:
# Get the credentials:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

#geo_locations = '-122.75,36.8,-121.75,37.8,-74,40,-73,41'

# Coordinates of berlin

lon, lat, R =13.404954, 52.520008, 50.0
geo_locations = gps.bounding_square_coordinates(lon,lat,R)

# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN_KEY'], creds['ACCESS_TOKEN_SECRET'],
                   long=lon,lat=lat,Radius=R)
# Start the stream
stream.statuses.filter(locations=geo_locations)  
#stream.statuses.filter(track=trackstring)  

query info:  52.520008 13.404954
Time Zone:  None
['1022559961027629057', '', 52.520008, 13.404954, 50.0, 'Thu Jul 26 19:11:07 +0000 2018']
query info:  52.520008 13.404954
Time Zone:  None
['1022560216884211712', '', 52.520008, 13.404954, 50.0, 'Thu Jul 26 19:12:08 +0000 2018']
query info:  52.520008 13.404954
Time Zone:  None
['1022560251990745094', '', 52.520008, 13.404954, 50.0, 'Thu Jul 26 19:12:16 +0000 2018']
query info:  52.520008 13.404954
Time Zone:  None
['1022560538683949056', '', 52.520008, 13.404954, 50.0, 'Thu Jul 26 19:13:25 +0000 2018']
query info:  52.520008 13.404954
Time Zone:  None
['1022560806356049929', '', 52.520008, 13.404954, 50.0, 'Thu Jul 26 19:14:29 +0000 2018']
query info:  52.520008 13.404954
Time Zone:  None
['1022560826127990786', '', 52.520008, 13.404954, 50.0, 'Thu Jul 26 19:14:33 +0000 2018']
query info:  52.520008 13.404954
Time Zone:  None
['1022560991870177281', '', 52.520008, 13.404954, 50.0, 'Thu Jul 26 19:15:13 +0000 2018']
query info:  52.5200

KeyboardInterrupt: 